In [1]:
import numpy as np
import scipy
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
%matplotlib qt5
import testStreamingSvd
import streamingSvd as algo

In [2]:
# generate scalar AR(p) data
def generateARp(x_init, coeffs, num_steps, noise=0):
    p = x_init.shape[0]
    xs = np.append(x_init, np.zeros((num_steps)))
    for t in range(num_steps):
        eps = np.random.normal(loc=0, scale=noise)
        xs[t+p] = np.dot(coeffs, xs[t:t+p]) + eps
    return xs


# generate vector AR(1) data, init with last column of xs
def generateVAR1(xs, A, num_steps, noise=0):
    n, t = xs.shape
    xs = np.append(xs, np.zeros((n, num_steps)), axis=1)
    for curr_step in range(t, t+num_steps):
        eps = np.random.normal(loc=0, scale=noise, size=n)
        xs[:, curr_step] = np.dot(A, xs[:, curr_step-1]) + eps
    return xs


# get stable (stationary) coeffs for VAR(1) - max eigval < 1
def get_stable_A(n, low=0, upp=2):
    unstable = True
    while unstable:
        A = np.random.uniform(low, upp, size=(n,n))
        evals, _ = np.linalg.eig(A)
        if np.max(np.abs(evals)) < 1:
            unstable = False
    return A


# rotate 3x3 matrix around x axis by angle theta
def rotate_3dx(A, theta):
    rot = np.array([[1, 0, 0], 
                    [0, np.cos(theta), -np.sin(theta)], 
                    [0, np.sin(theta), np.cos(theta)]])
    return np.dot(rot, A)


# plot 2d vecs
def plot_2d(A, ax=None):
    if ax is None:
        fig, ax = plt.subplots(1)
    orig = np.zeros(shape=(A.shape))
    ax.quiver(*orig, A[:, 0], A[:, 1], angles='xy', scale_units='xy', scale=1)
    xlim = np.max(np.abs(A[0, :])) + .5
    ylim = np.max(np.abs(A[1, :])) + .5
    ax.set_xlim(-xlim, xlim)
    ax.set_ylim(-ylim, ylim)
    return ax


def plot_plane3d(normal, point, ax_lim, ax):
    # a plane is a*x+b*y+c*z+d=0
    # [a,b,c] is the normal. Thus, we have to calculate
    # d and we're set
    d = -point.dot(normal)
    # create x,y
    pl_lim = np.ceil(ax_lim).astype('int')
    xx, yy = np.meshgrid(range(-pl_lim, pl_lim), range(-pl_lim, pl_lim))
    # calculate corresponding z
    z = (-normal[0] * xx - normal[1] * yy - d) * 1. /normal[2]
    # plot the surface
    ax.plot_surface(xx, yy, z, alpha=.25)
    

In [64]:
# visualizing timescale of subspace (plane) rotation
num_pointsA = 200
num_pointsB = 1 * num_pointsA
unif_lim = 1
noise_sig = 0.01

plane_axA = np.random.normal(loc=1, scale=noise_sig, size=(1,num_pointsA))
A = np.random.uniform(-unif_lim, unif_lim, size=(2,num_pointsA))
A = np.insert(A, 0, plane_axA, axis=0)
plane_axB = np.random.normal(loc=-1, scale=noise_sig, size=(1,num_pointsB))
B = np.random.uniform(-unif_lim, unif_lim, size=(2,num_pointsB))
B = np.insert(B, 1, plane_axB, axis=0)
Atot = np.append(A, B, axis=1)
# Atot = np.stack((A,B), 2).reshape(A.shape[0], -1)  # mixing up order

k = 2
l1 = 200
l = 5
decay = .95

num_iters = np.ceil((Atot.shape[1] - l1) / l).astype('int')
Qt, S, Qcoll = algo.getSvd(Atot, k, l1, l, num_iters, decay_alpha=decay)

# %matplotlib qt5
fig = plt.figure()
ax1 = fig.add_subplot(121, projection='3d')
ax_lim = np.max(np.abs(Atot))
ax1.set(xlim=(-ax_lim, ax_lim), ylim=(-ax_lim, ax_lim), zlim=(-ax_lim, ax_lim))
planeA = Atot[:, :num_pointsA]
planeB = Atot[:, num_pointsA:]
ax1.scatter(planeA[0,:], planeA[1,:], planeA[2,:], alpha=.4, 
            label='first {} cols'.format(num_pointsA))
ax1.scatter(planeB[0,:], planeB[1,:], planeB[2,:], alpha=.4, 
            label='last {} cols'.format(num_pointsB))
ax1.legend()

orig = np.zeros((3,1))
colors = np.zeros((num_iters, 3))
colors[:, 0] = np.arange(0, 1, 1/num_iters)
colors[:, 2] = np.arange(0, 1, 1/num_iters)
for i in range(Qcoll.shape[2]):
    Q = Qcoll[:, :, i]
    ax1.quiver(*orig, Q[0,:], Q[1,:], Q[2,:], color=colors[i, :])

ax2 = fig.add_subplot(122)
Qvecs = Qcoll[:, 0, :]
Qvecs_dists = Qvecs[:, 1:] - Qvecs[:, :-1]
Qvecs_dists = np.linalg.norm(Qvecs_dists, axis=0)
ax2.plot(Qvecs_dists)
ax2.set(xlabel='iter (each iter adds {} cols of A)'.format(l), 
        ylabel='distance between subsequent 1st sing vecs', 
        title='subspace rotation \"inertia\" (decay={:.2f})'.format(decay))

In [3]:
n = 2
numsteps1 = 100
numsteps2 = 1000
sigma = .1

# A1 almost diagonal, 0 out last entry
A1 = np.diag(np.random.uniform(0, 1, size=n))
A1[n-1, n-1] = 0
x_init = np.random.uniform(-1, 1, size=(n, 1))
xs = generateVAR1(x_init, A1, numsteps1, noise=sigma)
 
# nice oscillations
A2 = np.array([
       [ 0.9767596 , -0.28857368,  0.4491203 ],
       [ 0.97991711,  0.49943434,  0.32865101],
       [ 0.59191832, -0.16951207,  0.57953176]])
A2 = A2[:n, :n]
# A2 = get_stable_A(n, upp=1)
xs = generateVAR1(xs, A2, numsteps2, noise=sigma)

%matplotlib qt5
plt.plot(xs.T)
plt.axvline(numsteps1, color='k')

# variance
U1, S1, _ = np.linalg.svd(xs[:, :100])
U2, S2, _ = np.linalg.svd(xs[:, 102:])
print(S1)
print(S2)

In [45]:
# %matplotlib qt
n = 3
num_steps = 100
num_steps2 = 100
sigma = 0.05

A1 = np.zeros((3,3))
A1[:2, :2] = get_stable_A(2, low=-3, upp=3)
A1[2, 2] = 1

A2 = np.zeros((3,3))
A2[1:, 1:] = get_stable_A(2, low=-3, upp=3)
A2[0, 0] = 1

x_init = np.random.uniform(1, 1, size=(n,1))
xs = generateVAR1(x_init, A1, num_steps, noise=sigma)
xs = generateVAR1(xs, A2, num_steps2, noise=sigma)
fig = plt.figure()
ax = fig.add_subplot(121)
ax.plot(xs.T)

U, S, _ = np.linalg.svd(xs)
U = np.multiply(U, S)
U = np.multiply(U, .1/np.max(U))


ax = fig.add_subplot(122, projection='3d')
colors = np.zeros((xs.shape[1], 3))
colors[1:, 2] = 1
ax.scatter(xs[0, :], xs[1, :], xs[2, :], c=colors)
ax.plot(xs[0, :], xs[1, :], xs[2, :])
# orig = np.zeros((3,1))
# ax.quiver(*orig, U[0,:], U[1,:], U[2,:], color='r')

# plane through first two left sing vecs
ax_lim = np.max(np.abs(xs))
# point  = U[:, 0]
# normal = U[:, 2]
# plot_plane3d(normal, point, ax_lim, ax)

ax.set(xlim=(-ax_lim, ax_lim), 
       ylim=(-ax_lim, ax_lim), 
       zlim=(-ax_lim, ax_lim))

print(A1, '\n--')
print(A2)

[[-1.75944709 -2.1525221   0.        ]
 [ 1.40324527  1.73502454  0.        ]
 [ 0.          0.          1.        ]] 
--
[[ 1.          0.          0.        ]
 [ 0.         -1.67023649 -1.77291978]
 [ 0.          2.41801908  2.05132449]]
